#### ```0. Basic_check(df)```
- pd.sample 확인, info, 전체 데이터 수 및 null 값 현황을 확인하는 함수  
---

### 1. 날짜 관련 함수 처리
#### ```1-1. date_preprocess(df, date_colname, time_colname, new_colname)```
- tran = tran_date + tran_time
- play = play_date + play_st_time
- 시간 데이터 형식을 4자리 수로 변환 ex) 615->0615
- 날짜와 시간을 합쳐서 datetime 형식 데이터로 변경함
- 각 날짜별 요일을 한국어로 추출하여 tran_day, play_day col로 추가
- 함수 생성 후, 테스트를 위해 ```1000개``` 데이터만 사용함(df1000)
---
#### ```1-2. float_to_datetime(df, col_list)```
- float 타입의 데이터들을 datetime(yyyy-mm-dd 형식)으로 변경
- pre_open_date, open_date를 col_list로 전달
---
#### ```1-3. tran_rank_min(df)```
- 'performance_code'로 그룹화하여 'tran' 열을 가장 빠른 순서대로 정렬 후, rank를 매김
- 최종적으로 사용한 method = 'min' 이지만, method = 'first'와 비교하여 확인한 과정도 남겨두었음. 
- 특정 공연에서 빨리 예매된 좌석이 어디인지를 파악하고 그 좌석이 좋은 좌석이라는 feature로 사용하기 위한 작업
---
### ```2. 데이터 나누기```
### ```2-1. separate_seat(df)```
- 좌석 정보 분리하여 4개의 col에 저장 : seat_floor(층),seat_block(블록),seat_col(열),seat_num(좌석번호)
### ```2-2. discounts(df)```
- 할인명, 할인률 구분하기   
---
### `3. 오류 데이터 처리`
- member_yn = 'N'인 데이터 중에서 멤버십, 젠더 정보가 있는 경우 'Y'로 변경`
---
### `4. 데이터 이진화`
- pre_open_date, gender, member_yn 데이터 2진화 수행, gender의 경우 nan을 2로 labeling  
** pre_open_date : 예술의전당 유료회원 상대로 미리 예매를 할 수 있는 하루 전 혜택, 기획자/대관자의 선택에 의해 결정  

---
### `5. membership column 정리`
- 6개의 membership_type 컬럼을 membership_free, membership_paid 2개의 컬럼으로 정리  
- free : 노블, 싹틔우미, 일반  
- paid : 법인, 골드, 블루, 그린, 무료로 등급을 나누어 컬럼 간소화  

### `추후 작업사항`
- Nan값 처리 [age, gender, 멤버십들, discount_type, open_date, pre_open_date, running_time, intermission]  
- dtype 변경

In [1]:
# !pip install -r requirements.txt
# !pip install pandas matplotlib seaborn

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings

plt.rc('font', family = 'Malgun Gothic')
filterwarnings('ignore')

# 전체 데이터 출력 및 확인 옵션 설정
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_info_columns

100

# 0. basic_check
샘플 데이터 print, info, 전체 데이터 수 및 null 값 현황을 확인하는 함수

In [3]:
def basic_check(df):
    # 샘플 데이터 10개 확인
    print('샘플 데이터 10개 확인\n', df.sample(10))
    print('==================================================')
    # info 확인
    print('info 확인\n', df.info())
    print('==================================================')
    # 전체 데이터 수 및 null 값 확인
    print('전체 데이터 수 : ', len(df)) 
    print('null 값 확인\n', df.isnull().sum())

# 1-1. date_preprocess
* input
  - df : 데이터프레임
  - date_colname : 날짜 정보가 담긴 컬럼명 (yyyymmdd)
  - time_colname : 시간 정보가 담긴 컬럼명 (hhmm), 시간, 분을 4자리 수로 변환하는 작업 필요 ex) 615->0615
  - new_colname : 새롭게 생성할 컬럼명, date_colname + time_colname
* output
  - df : 데이터프레임
  - new_colname : date_colname + time_colname를 합친 후, datetime 형태로 가공(yyyy-mm-dd hh:mm:ss)
  - new_colname_day : 새롭게 생성된 컬럼에서 요일 정보를 한국어로 추출

In [5]:
def date_preprocess(df, date_colname, time_colname, new_colname):
    df_new = df.copy()
    
    # 먼저 시간, 분을 4자리 수로 변환 ex) 615->0615
    df_new[time_colname] = df_new[time_colname].astype(str).str.zfill(4)
    df_new[date_colname] = df_new[date_colname].astype(str)
    df_new[new_colname] = df_new[date_colname] + df_new[time_colname]
    # datetime화
    df_new[new_colname] = pd.to_datetime(df_new[new_colname], format='%Y%m%d%H%M')
    # 요일 만들기
    df_new[f'{new_colname}_day'] = df_new[new_colname].dt.day_name(locale='ko_kr') #locale 미설정시 영어로 나옴
    #기존 열들은 삭제
    df_new = df_new.drop([date_colname, time_colname], axis=1)
    return df_new

In [ ]:
# # TEST with 1000개 데이터
# df1000 = df[:1000]
# ## 1. tran = tran_date + tran_time
# df1000 = date_preprocess(df1000, 'tran_date', 'tran_time', 'tran')

# ## 2. play = play_date+ play_st_time
# df1000 = date_preprocess(df1000, 'play_date', 'play_st_time', 'play')
# df1000

# 1-2. float_to_datetime
- float 타입의 데이터들을 datetime(yyyy-mm-dd 형식)으로 변경
- pre_open_date, open_date

In [7]:
def float_to_datetime(df, col_list):
    for col in col_list:
        df[col] = pd.to_datetime(df[col], format='%Y%m%d')

# 1-3. tran_rank_min(df)
- 'performance_code'로 그룹화하여 'tran' 열을 가장 빠른 순서대로 정렬 후, rank를 매김
- 최종적으로 사용한 method = 'min' 이지만, method = 'first'와 비교하여 확인한 과정도 남겨두었음. 

In [9]:
# rank method = first
# def tran_rank_first(df):    
#     df['tran_rank'] = df.groupby('performance_code')['tran'].rank(ascending=True, method='first')

In [ ]:
# df1000.query("`performance_code`==1764")
# 순위가 매겨진 걸 확인해봤는데, method = 'first'를 사용하면 먼저 나타난 데이터에 우선순위를 부여함.
# 그러나 tran이 같은(즉, 동시에 예매한 경우) 경우가 존재. 
# 이 과정은 특정 공연에서 빨리 예매된 좌석이 어디인지를 파악하고 그 좌석이 좋은 좌석이라는 feature로 사용하고자 하므로
# method = 'dense'을 사용해서 1,2,3이 아니라 1,1,2과 같이 순위를 매기기로 하였음.

In [12]:
# rank method = min
def tran_rank_dense(df):    
    df['tran_rank'] = df.groupby('performance_code')['tran'].rank(ascending=True, method='dense')

# 2-1. separate_seat(df)
- 좌석 정보 분리하여 4개의 col에 저장 : seat_floor(층),seat_block(블록),seat_col(열),seat_num(좌석번호)  
- ChatGPT를 활용해 얻은 정규표현식 설명
  - df['seat'].str.extract(r'(\d+)층\s*([A-Z]*\w*)?(\d+열)?\s*(\d+)'):  
  - str.extract 메서드는 문자열을 추출하기 위한 Pandas Series의 문자열 메서드입니다.  
  - r'(\d+)층\s*([A-Z]*\w*)?(\d+열)?\s*(\d+)'는 정규식 패턴을 나타냅니다.  
  - (\d+)는 1개 이상의 숫자를 나타냅니다. 이 부분은 seat_floor (층)와 seat_num (좌석번호)에 해당합니다.  
  - 층 문자열은 입력 문자열에서 "층"이라는 글자를 정확히 일치시키는 역할을 합니다.  
  - ([A-Z]*\w*)?는 영대소문자와 숫자로 이루어진 문자열을 나타냅니다. 이 부분은 seat_block (블록)에 해당합니다.  
  - (\d+열)?는 1개 이상의 숫자 뒤에 "열"이라는 문자열이 나타날 수 있는 것을 나타냅니다. 이 부분은 seat_col (열)에 해당합니다.  
  - \s*는 공백 문자 (스페이스)가 0개 이상 나타날 수 있는 것을 나타냅니다. 이렇게 함으로써 숫자와 문자열 사이에 여분의 공백이 있어도 처리할 수 있습니다.  

In [14]:
# df1000.seat.unique()
# unique 값을 확인해본 결과, 크게 아래 5가지 유형이 파악됨
# 3층 BOX9 10
# 2층 BOX4 1열 13
# 1층 7열 5
# 1층 C블록 16열 3
# 3층 G블록5열 8

array(['3층 BOX9 10', '1층 7열 5', '1층 C블록 16열 3', '1층 2열 3', '1층 B블록12열 7',
       '1층 A블록2열 1', '3층 E블록4열 8', '2층 D블록8열 4', '1층 B블록 11열 9',
       '1층 C블록17열 3', '2층 B블록 4열 3', '3층 BOX9 5', '1층 B블록 10열 6',
       '3층 BOX12 3', '2층 2열 5', '2층 D블록8열 3', '1층 C블록2열 2', '2층 E블록3열 16',
       '2층 BOX2 2', '1층 C블록20열 4', '1층 D블록10열 8', '1층 A블록 4열 2',
       '2층 BOX4 1열 13', '2층 3열 9', '1층 A블록 3열 2', '1층 4열 1', '1층 B블록7열 7',
       '3층 G블록5열 8', '1층 B블록21열 9', '1층 B블록17열 4', '1층 B블록 15열 2',
       '1층 D블록18열 6', '2층 A블록 6열 3', '3층 M블록1열 2', '1층 B블록 16열 9',
       '1층 8열 18', '1층 B블록 10열 4', '2층 B블록 1열 6', '1층 C블록15열 8',
       '1층 B블록3열 7', '2층 3열 19', '1층 B블록 11열 1', '3층 F블록5열 5',
       '2층 B블록 3열 3', '1층 E블록17열 1', '2층 B블록 2열 6', '1층 D블록7열 11',
       '1층 B블록 16열 7', '1층 7열 15', '1층 9열 13', '1층 D블록22열 3', '1층 4열 14',
       '3층 G블록1열 5', '1층 C블록4열 6', '1층 B블록7열 5', '1층 C블록 15열 5',
       '1층 8열 16', '1층 E블록19열 3', '2층 3열 6', '3층 F블록4열 3', '1층 C블록6열 7',
       '2층 C블록7열 7', '2층 A블록4열 7', '1층 

In [15]:
def separate_seat(df):    
    df[['seat_floor', 'seat_block', 'seat_col', 'seat_num']] = df['seat'].str.extract(r'(\d+)층\s*([A-Z]*\w*)?(\d+열)?\s*(\d+)')
    #seat_block에 공백기준 분리가 안 되어 있어서 'C블록4열', '3열'과 같이 붙어있던 데이터 제거
    df['seat_col'] = df['seat_block'].str.extract(r'(\d+열)', expand=False)
    df['seat_block'] = df['seat_block'].str.replace(r'\d+열', '', regex=True)
    #마지막으로 seat_block에서는 '블록', seat_col에서는 '열'이라는 글자 제거하기
    df['seat_block'] = df['seat_block'].str.replace('블록', '', regex=True)
    df['seat_col'] = df['seat_col'].str.replace('열', '', regex=True)
    df_new = df.drop(['seat'], axis=1)
    return df_new

# 2-2. discount_type_sep()
- 일반 : 0%  
- 초대권 : 100% , 유의사항 : [초대권] 이런 게 있음.. '초대'라는 단어를 포함하고 있으면 항상 100% 할인일까? 확인 예정
- 그 외 : 각자 %
- nan : 기획사 등.. 지금 시점에서 정확한 %를 확인할 수 없는 경우

In [ ]:
# df['discount_type'].unique()

In [18]:
def discount_type_sep(df):
    # discount_name과 discount_rate 열 초기화
    df['discount_name'] = np.nan
    df['discount_rate(%)'] = np.nan

    # 조건에 따라 discount_name과 discount_rate 설정
    for idx, row in df.iterrows():
        if '일반' in row['discount_type']:
            df.at[idx, 'discount_name'] = '일반'
            df.at[idx, 'discount_rate(%)'] = '0'
        elif '초대권' in row['discount_type']:
            df.at[idx, 'discount_name'] = '초대권'
            df.at[idx, 'discount_rate(%)'] = '100'
        elif '%' in row['discount_type']:
            rate = ''.join(filter(str.isdigit, row['discount_type']))
            df.at[idx, 'discount_name'] = '기타'
            df.at[idx, 'discount_rate(%)'] = rate + ''

    # discount_type 열 제거
    df.drop('discount_type', axis=1, inplace=True)

# 1. memeber_yn == 'N' 오류 처리  
member_yn = 'N' 인데 멤버십, 젠더 정보가 있는 경우가 있음  
멤버십, 젠더의 정보가 있다는 것은 회원이라는 것을 의미하므로 Y로 되어야 함

In [ ]:
def error_member_yn(df):
    idx = df.query('`member_yn` == "N" and `gender` == `gender`').index
    df.loc[idx,'member_yn'] = 'Y'
    
    return df

# 2. data labeling  
  
  * pre_open_date  
    값이 있다면(선예매 있는 공연) 1, 값이 없다면(선예매 없는 공연) 0

  * gender  
    여자는 1, 남자는 0, Nan은 2로 Labeling

  * member_yn
    Y는 1로 N 은 0

In [ ]:
def labeling(df):
    
    # 1. pre_open_date 
    x = np.array(df.pre_open_date) # pre_open_date 값들을 numpy array배열로 변환 
    df.pre_open_date = np.where(np.isnan(x), 0, 1) # 값이 존재하면 1(선예매 O), 없으면 0(선예매 X) 으로 변환

    # 2. gender
    n = len(df)
    label = []
    for i in range(n):
        if df['gender'][i] == 'M':
            label.append(0)
        elif df['gender'][i] == 'F':
            label.append(1)
        else:
            label.append(2)
            
    df.gender = label
            
    # 3. member_yn
    x = np.array(df.member_yn)
    df.member_yn = np.where(x == 'Y', 1, 0)

    return df

## 3. membership column 정리  

In [ ]:
 def membership_col_make(df):
    
    #data를 member, nonmember를 나눠줌
    member_df = df.query('`member_yn` == 1')
    nonmember_df = df.query('`member_yn` == 0')
    
    # nonmember free 및 paid 비회원 처리
    nonmember_df['membership_free'] = '비회원'
    nonmember_df['membership_paid'] = '비회원'
    
    # free, paid 타입 구분
    free = ['노블', '싹틔우미', '무료'] # 무료는 알고리즘 내에서 '일반'으로 처리함
    paid = ['법인', '골드', '블루', '그린'] # 다 해당하지 않으면 무료, index가 작을수록 높은등급
    
    membership_free = []
    membership_paid = []
    # 6번 type은 모두 nan이므로 제외 
    membership_list = member_df[['age', 'membership_type_1','membership_type_2','membership_type_3','membership_type_4','membership_type_5']].values
    
    # 일반 25 이상(직장인), 싹틔우미 ~24(대학생 이하), 노블 69~
    for i in range(len(membership_list)):
        tmp_free = []
        tmp_paid = []
        m_types = membership_list[i]
        
        # 5개의 멤버십 타입중, free에 해당하는 것들을 tmp_free에 넣고 나머지는 tmp_paid에 넣음
        for j in range(1, 6):
            if m_types[j] in ['노블', '무료', '싹틔우미']:
                tmp_free.append(m_types[j])
                
            else:
                tmp_paid.append(m_types[j])
                
        # paid의 nan 중복제거
        tmp_paid = set(tmp_paid)
        
        # '무료'만 있거나 노블, 싹틔우미가 동시에 있는 경우
        if len(tmp_free) == 1 or len(tmp_free) == 3:
            # 나이정보를 결합해 입력
            if m_types[0] < 20:
                membership_free.append('싹틔우미')
                
            elif m_types[0] == 20:
                if len(tmp_paid) > 1:
                    membership_free.append('일반')
                    
                else:
                    membership_free.append('싹틔우미')
                    
            elif m_types[0] < 70:
                membership_free.append('일반')
            
            # 만약 나이정보가 없다면 '처리'를 넣어 나중에 추가처리
            else:
                if m_types[0] == m_types[0]:
                    membership_free.append('노블')
                else:
                    membership_free.append('처리')
                    
        # 이상이 없는 경우 주어진 값대로 넣어줌
        else:
            if tmp_free[0] != '무료':
                membership_free.append(tmp_free[0])
            else:
                membership_free.append(tmp_free[1])
        
        
        if len(tmp_paid) == 1: # Nan만 있다면 => 순수 무료회원
            membership_paid.append('무료')
        
        # Nan이 없다면 queue구조를 활용해 높은 등급의 멤버십만 남겨 데이터에 추가
        else: 
            queue = list(tmp_paid)
            ranking = 4
            
            while len(queue) > 1:
                x = queue.pop(0)
                
                if x in paid:
                    r = paid.index(x)
                    if r < ranking:
                        ranking = r
                        queue.append(x)
            
            membership_paid.append(queue[0])

    member_df['membership_free'] = membership_free
    member_df['membership_paid'] = membership_paid
    
    #member, nonmember 병합
    result = pd.concat([member_df, nonmember_df], axis = 0)
    
    return result.sort_index()

# 작성예정

In [20]:
# column들의 데이터 타입을 int로 변경하는 함수
def dtype_to_int(df, col_list):
    for col in col_list:
        df[col] = df[col].astype('int')
        
# dtype_to_int(df, ['age', 'pre_open_date', 'open_date'])